# Data Preparation
This notebook copies the invoices into a ./data folder, scans them with ocr and combines it with some structured data in a csv file

In [2]:
import json
import os
import re
import shutil
from tqdm.notebook import tqdm

import pandas as pd
import pyodbc
import pytesseract
from pdf2image import convert_from_path
from pdf2image.exceptions import PDFPageCountError

In [3]:
path = '//srvaw03/AF-Drive/BACH/Vendor/B'
pattern = '^PI20_[0-9]{6}$'
pattern1 = '^VE20_[0-9]*$'
data_fields = 'nummer,sumbrutto,sumnetto,lief2'
data_path = './data'
config = json.load(open('config.json'))

## Get Files

In [ ]:
def get_files(path):
    for folder in os.listdir(path):
        print(folder)
        if re.match(pattern, folder) and len(os.listdir(path + "/" + folder)) > 0 and folder not in os.listdir(
                data_path):
            for file in os.listdir(path + "/" + folder):
                os.mkdir(data_path + '/' + folder)
                shutil.copyfile(path + "/" + folder + "/" + file, data_path + "/" + folder + "/" + file)
        elif os.path.isdir(path + "/" + folder) and (
                re.match(pattern1, folder) or folder == 'invoices' or folder == '_archive'):
            get_files(path + "/" + folder)

In [ ]:
get_files(path)

## OCR

In [13]:
t = tqdm(os.listdir(data_path))
for folder in t:
    t.set_description(folder)
    t.refresh()
    if not os.path.isfile(data_path + '/' + folder + '/ocr.txt'):
        for file in os.listdir(data_path + "/" + folder):
            try:
                images = convert_from_path(data_path + "/" + folder + "/" + file)
                file_string = ''
                for image in images:
                    file_string += pytesseract.image_to_string(image)
                with open(data_path + "/" + folder + "/ocr.txt", "w") as f:
                    f.write(file_string)
            except PDFPageCountError:
                shutil.rmtree(data_path + "/" + folder, ignore_errors=True)
                continue

  0%|          | 0/454 [00:00<?, ?it/s]

## Get Data

In [8]:
def get_data():
    server = config['server']
    database = config['database']
    username = config['db_user']
    password = config['db_password']
    cnxn = pyodbc.connect(
        'DRIVER={SQL Server};SERVER=' + server + ';DATABASE=' + database + ';UID=' + username + ';PWD=' + password)
    query = 'SELECT {} FROM abasbi.dbo.Purchasing$Purchasing_4_1 where MANDANT_ID = 2 AND NUMMER in ({})'
    numbers = ["'" + i[5:] + "'" for i in os.listdir(data_path)]
    query = query.format(data_fields, ','.join(numbers))
    df = pd.read_sql(query, cnxn)
    duplicates = set([x for x in df["nummer"].tolist() if df["nummer"].tolist().count(x) > 1])
    if len(duplicates) > 0:
        for folder in os.listdir(data_path):
            if folder[5:] in duplicates:
                shutil.rmtree(data_path + "/" + folder, ignore_errors=True)
        get_data()
    else:
        df.to_csv('data.txt')

In [15]:
get_data()

## Merge Data with text

In [16]:
df = pd.read_csv('data.txt')
df = df.sort_values('nummer')
text = []
for folder in os.listdir(data_path):
    tmp_path = data_path + "/" + folder + "/ocr.txt"
    with open(tmp_path) as file:
        text.append(file.read())
df['text'] = text
df.to_csv('train.txt')